In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
import numpy as np
import os
import time

In [3]:
text = open('shakespeare.txt').read()
print('Total characters: %d' %(len(text)))

Total characters: 1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(list(set(text)))
print('Total vocab: %d' %(len(vocab)))

Total vocab: 65


In [6]:
int_to_char = np.array(vocab) # a numpy series
print(int_to_char)
print()
print(int_to_char.shape)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']

(65,)


In [7]:
char_to_int = dict((ch,i) for i,ch in enumerate(vocab)) # a dict
# int_to_char = dict((i,ch) for i,ch in enumerate(vocab))

In [8]:
# char_to_int

In [9]:
# converting input format from char to int

text_as_int = np.array([char_to_int[ch] for ch in text])
text_as_int.shape

(1115394,)

In [10]:
text_as_int [:100]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [11]:
# Tensors can be explicitly converted to NumPy ndarrays by invoking the .numpy() method on them

seq_length = 100
examples_per_epoch = len(text)//seq_length  # 11153; number of sequences per epoch

# tf.data.Dataset.from_tensor_slices function converts the text vector into a stream of character indices
# it converts your numpy dataset into tensorflow dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for char in char_dataset.take(5):
#     print(char.numpy())
    print(int_to_char[char.numpy()])
type(text_as_int) # <class 'numpy.ndarray'>

F
i
r
s
t


numpy.ndarray

In [12]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
type(sequences)
for char in sequences.take(3):
    print(repr(''.join(int_to_char[char.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"


In [14]:
sequences

<DatasetV1Adapter shapes: (101,), types: tf.int64>

In [15]:
def split_into_input_target(chunk):
    inputs = chunk[:-1]
    targets = chunk[1:]
    return inputs, targets

dataset = sequences.map(split_into_input_target)

In [16]:
for inputs, targets in dataset.take(1):
    print('Input data:', repr(''.join(int_to_char[inputs.numpy()])))
    print('Target data:', repr(''.join(int_to_char[targets.numpy()])))

Input data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [17]:
for i, (input_idx, target_idx) in enumerate(zip(inputs[:5], targets[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(int_to_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(int_to_char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [18]:
examples_per_epoch

11153

# Create training batches

In [19]:
# Batch size

batch_size = 64
steps_per_epoch = examples_per_epoch//batch_size # no. of batches per epoch ; 174

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements)
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build the Model

In [20]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [21]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# Try the model

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    examples_batch_predictions = model(input_example_batch)
    print(examples_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [27]:
# For the first example in the batch

sample_indices = tf.random.categorical(examples_batch_predictions[0], num_samples=1)

# Given a tensor input, 'tf.squeeze' returns a tensor of the same type
# with all dimensions of size 1 removed. If you don't want to remove all
# size 1 dimensions, you can remove specific size 1 dimensions by specifying axis
# e.g.- here last tensor is being removed

# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t))  # [2, 3]
# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t, [2, 4]))  # [1, 2, 3, 1]
# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t, [-1]))  # [1, 2, 1, 3, 1]
sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()

In [35]:
# This gives us, at each timestep, a prediction of the next character index:

sample_indices # (100, )

array([64, 30, 52, 59, 49,  7, 54, 59,  5, 61, 13, 30, 35, 46, 53, 44, 35,
       34, 38, 13, 12, 43, 39, 58, 55,  3, 22, 51, 38, 60, 44, 61, 33, 59,
       19, 14, 43, 12, 38, 50, 17, 57, 64, 40, 20, 22, 28, 24, 49, 10, 22,
       12, 62, 41, 51, 31, 62, 39, 19, 36, 20, 59, 19, 61, 52,  6, 51,  6,
       51, 12, 35, 45, 59, 61, 48, 59, 58, 50, 45, 31, 16,  9, 35, 14, 13,
       20, 63, 56, 17, 51, 56,  8, 42, 38, 26,  0, 39, 34, 61,  1])

In [31]:
print("Input:", repr("".join(int_to_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(int_to_char[sample_indices])))

Input: 'the state, who care for you like fathers,\nWhen you curse them as enemies.\n\nFirst Citizen:\nCare for u'

Next Char Predictions: 
 "zRnuk-pu'wARWhofWVZA?eatq$JmZvfwUuGBe?ZlEszbHJPLk:J?xcmSxaGXHuGwn,m,m?WguwjutlgSD3WBAHyrEmr.dZN\naVw "


# Train the model

In [42]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  
example_batch_loss = loss(target_example_batch, examples_batch_predictions)
print('Prediction shape: ', example_batch_loss.shape, ' # (batch_size, sequence_length)')
print('scalar_loss: ',example_batch_loss.numpy().mean())

Prediction shape:  (64, 100)  # (batch_size, sequence_length)
scalar_loss:  4.1746407


In [43]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss=loss)

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

In [46]:
# steps_per_epoch = 174
epochs = 3
history = model.fit(dataset.repeat(), epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
174/174 [==============================] - 648s 4s/step - loss: 1.9266
Epoch 2/3
174/174 [==============================] - 654s 4s/step - loss: 1.6743
Epoch 3/3
174/174 [==============================] - 663s 4s/step - loss: 1.5343


# Generate text

In [47]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints/ckpt_3'

In [48]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [50]:
def generate_text(model, start_string):
    # Number of chars to generate
    num_generate = 1000
    
    # Converting our start string to numbers (vectorizing)
    input_eval = [char_to_int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Empty string to store our results
    text_generated = []
    
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # Using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(int_to_char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [54]:
# print(generate_text(model, start_string=u"ROMEO: "))
print(generate_text(model, start_string="ROMEO: "))

ROMEO: the spor's vail, make do well I come to need
Aboule; and three be or Yenter.

ANGELO:
Then we dare an one as the dukbori'g with
Which I hip soulled soldier the revelles counternands and upon
These was drew theque not she to Tourtht dear.
Methit what is not tlould so be fanch.

Busting:
What high you already; and they ale face come,
Made the fant of but whengether with carrel them, while own joys, fear too sworth;
Grief this sway sid takes in their brot
Ungear op intled, ans him it see Mantinglan:
My pause, let this dusicibuling black
Fortwness be uncevuty concius
And our parkly, sin, there you with nand is calls
Sdeak as for such and at the winddal friend-cousin,
I am not to excide,
Whither pley'st are not put you; ever'd one--

MENENIUS:
Being on the rayeits from me,
Nare with that Accounsee what, door hathed should
hear forlest and beckn beit my tates.

GOND:
Bidlanced on, and this bauster for so.
That more your loss done not rief in intrunity.
Will be this well mean loar, he'